In [27]:
import requests
import pandas as pd
import random
import time
from io import StringIO
from bs4 import BeautifulSoup

In [28]:
def getTeamUrls(stats_url):
    start = time.time()
    data = requests.get(stats_url)
    delay = time.time() - start
    time.sleep(random.randint(40, 75)*delay)

    soup = BeautifulSoup(data.text)
    afc_standings = soup.select('table.stats_table')
    afc_team_links = []
    for tag in afc_standings:
        links = tag.find_all('a')
        afc_team_links.extend(links)
    afc_team_links = [link.get("href") for link in afc_team_links]
    
    
    
    nfc_standings = soup.select('#NFC')
    nfc_team_links = []
    for tag in nfc_standings:
        links = tag.find_all('a')
        nfc_team_links.extend(links)
    nfc_team_links = [link.get("href") for link in nfc_team_links]
    team_links = afc_team_links + nfc_team_links
    team_urls = [f"https://pro-football-reference.com{link}" for link in team_links]
    return team_urls

In [29]:
#Making pandas database out of Game Results and Basic Stats
def makeTeamDb (team_url):
    start = time.time()
    data = requests.get(team_url)
    delay = time.time() - start
    time.sleep(random.randint(40, 75)*delay)
    
    matches = pd.read_html(StringIO(data.text), match = "Schedule & Game Results Table")[0]
    
    #Adding names for unnamed columns manually
    matches = matches.rename(columns={'Unnamed: 3_level_1' : 'Time', 'Unnamed: 4_level_1' : 'Game Link', 'Unnamed: 5_level_1' : 'Result',
                                      'Unnamed: 8_level_1' : 'Home/Away'},)
    
    
    #Renaming columns 
    new_columns = []
    for column in matches.columns:
        if column[0] == 'Offense':
            new_columns.append('Off' + column[1])
                
        elif column[0] == 'Defense':
            new_columns.append('Def' + column[1])
    
        elif column[0] == 'Expected Points':
            new_columns.append('EP' + column[1][:3])
    
        elif column[0] == 'Score':
            new_columns.append(column[1] + 'Sc')
    
        else:
            new_columns.append(column[1])
    
    matches.columns = new_columns
    
    
    #Fixing empty data values
    matches['OT'] = matches['OT'].apply(lambda x : 'N' if pd.isna(x) or x=='N' else 'Y')
    matches['Result'] = matches['Result'].apply(lambda x : '' if pd.isna(x) or x=='' else x)
    matches['Rec'] = matches['Rec'].apply(lambda x : '' if pd.isna(x) or x=='' else x)
    matches['Home/Away'] = matches['Home/Away'].apply(lambda x : 'H' if pd.isna(x) or x=='H' else 'A')
    
    #Fixing empty data numbers to 0.0
    for column in matches.columns[13:]:
        matches[column] = matches[column].apply(lambda x : '0.0' if pd.isna(x) or x=='0.0' else x)
    
    #Fixing links to more detailed game stats
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text)
    game_table = soup.select('#games')
    game_links = []
    for tag in game_table:
        links = tag.find_all('a')
        game_links.extend(links)
    
    game_links = [link.get("href") for link in game_links]
    game_links = [link for link in game_links if '/boxscores' in link]
    game_links = [f"https://pro-football-reference.com{link}" for link in game_links]
    bye_week = 0
    while (matches['Opp'][bye_week] != 'Bye Week'):
        bye_week+=1
    game_links.insert(bye_week, '')
    matches['Game Link'] = game_links

    
    #Finding the team name and adding it to the table
    soup = BeautifulSoup(data.text)
    team_name = soup.find_all('h1')[0].find_all('span')[1].text
    matches['Team'] = team_name
    
    #Finally calculating and adding the year to the table
    year = team_url.split('/')[-1].replace('.htm', '')
    matches['Season'] = year
    return matches

In [34]:
years = list(range(2024,2021, -1))

In [31]:
#frame = makeTeamDb('https://www.pro-football-reference.com/teams/cle/2024.htm')
#frame

In [32]:
#code to save/cache all the html urls to this project in the html folder in case website goes down

#team_urls = getTeamUrls(stats_url)
#team_urls = ['https://www.pro-football-reference.com/teams/cle/2024.htm']


#for team_url in team_urls:
    #start = time.time()
    #data = requests.get(team_url)
    #delay = time.time() - start
    #time.sleep(random.randint(20, 60)*delay)
    #team_abr = team_url.split('/')[-2]
    #year = team_url.split('/')[-1].replace('.htm', '')[-2:]
    #with open ('./htmls/'+ team_abr + year + '.txt', 'w') as f:
        #f.write(data.text)

#with open ('./htmls/' + team_abr + '.txt', 'w') as f:
    #f.write("Goodbye")

In [40]:
all_matches = [] #list of dataframes of all the game data for each year for each team

for year in years: 
    stats_url = 'https://www.pro-football-reference.com/years/' + str(year) + '/index.htm'
    time.sleep(random.randint(40,75)) #extra sleep in between years to be safe
    team_urls = getTeamUrls(stats_url)
    for team_url in team_urls:
        print(team_url)
        db = makeTeamDb(team_url)
        all_matches.append(db)

    stats_url = stats_url.replace(str(year),str(year-1))



match_df = pd.concat(all_matches)
match_df.to_csv("matches.csv")
match_df

https://pro-football-reference.com/teams/buf/2024.htm
https://pro-football-reference.com/teams/mia/2024.htm
https://pro-football-reference.com/teams/nyj/2024.htm
https://pro-football-reference.com/teams/nwe/2024.htm
https://pro-football-reference.com/teams/pit/2024.htm
https://pro-football-reference.com/teams/rav/2024.htm
https://pro-football-reference.com/teams/cin/2024.htm
https://pro-football-reference.com/teams/cle/2024.htm
https://pro-football-reference.com/teams/htx/2024.htm
https://pro-football-reference.com/teams/clt/2024.htm
https://pro-football-reference.com/teams/jax/2024.htm
https://pro-football-reference.com/teams/oti/2024.htm
https://pro-football-reference.com/teams/kan/2024.htm
https://pro-football-reference.com/teams/sdg/2024.htm
https://pro-football-reference.com/teams/den/2024.htm
https://pro-football-reference.com/teams/rai/2024.htm
https://pro-football-reference.com/teams/phi/2024.htm
https://pro-football-reference.com/teams/was/2024.htm
https://pro-football-referen

ValueError: Length of values (20) does not match length of index (21)

In [39]:
stats_url = "https://www.pro-football-reference.com/years/2024/"
year = 2024
stats_url = stats_url.replace(str(year),str(year-1))
year -= 1
stats_url = stats_url.replace(str(year),str(year-1))
year -= 1
stats_url = 'https://www.pro-football-reference.com/years/' + str(year) + '/index.htm'

print(stats_url)

https://www.pro-football-reference.com/years/2022/index.htm
